In [269]:
import time
import math
import numpy as np
import scipy.io as sio
from numpy import linalg as la
#we use linear reg so phi so phi should not matter, however phi(x) does

In [270]:
#testing kernel for d=1
n=101
x_train = np.linspace(0,2*np.pi,101)
x_train = x_train[:,np.newaxis]
z_train = np.exp(x_train)
z_train = z_train.flatten()
z_test = z_train
d = 1 #dimension of the data
m = 0 #number of controls


In [271]:
#setting hyperparameters
rf_d = 100 #rf_d is dim of randomfeatures vector, to choose based on paper
mu = 0
sigma = 1
rf_mu = np.zeros(d)
rf_cov = 1/sigma * np.identity(d)
sigma_n = .1 #regularization parameter

In [272]:
samples = np.random.multivariate_normal(rf_mu,rf_cov,size =rf_d//2) #(rf_d//2,d)

In [273]:
#computing random features
#referring to random features as rf
#same sampling for different kernels
# s=(m+1)*rf_d

def rfquery_vector(x):    
    phi = np.empty(rf_d)
    dot_product = x @ samples.T
    phi[0::2]=np.sin(dot_product)
    phi[1::2]=np.cos(dot_product)
    phi = math.sqrt(2/rf_d) * phi
    return phi.flatten()   
                         
def computeZ(x_train): #(n,d)(n,m+1)
    phi = np.empty((n,rf_d))
    dot_product = x_train @ samples.T #(n,rf_d//2)
    phi[:,0::2]=np.sin(dot_product)
    phi[:,1::2]=np.cos(dot_product)
    phi = math.sqrt(2/rf_d) * phi #(n,rf_d)
    return phi
    

In [274]:
#computing compound kernel
#referring to compound as c
#using the same kernel

def compound_kernel(x_train):
    x_dif = x_train.reshape((n,1,d))-x_train
    kernel = np.exp(-np.sum(np.square(x_dif), axis=2)/(2 * sigma ** 2))
    return kernel

def cquery_vector(x_train, x_test):
    c_vector = np.exp(-la.norm(x_train-x_test, ord =2, axis=1)/(sigma ** 2))
    return c_vector

def compute_entry(x1,x2):
    return np.exp(-la.norm(x1-x2, ord=2)/sigma ** 2)

In [275]:
# random features training
# w = (Z'Z+simga^2I)^{-1}Z'y
Z = computeZ(x_train) #(n,s)
invZ_with_noise = la.inv(Z.T @Z + sigma_n ** 2 * np.identity((m+1)*rf_d)) #(s,s)
rf_w = invZ_with_noise @ Z.T @ z_train #(s,1)

In [276]:
# compound kernel training
# c_w = (K+sigma^2I)^{-1}K(x)
c_kernel = compound_kernel(x_train)
inv_ckernel_with_noise = la.inv(c_kernel + sigma_n**2 * np.identity(n))

In [277]:
rf_pred=np.empty(n)
c_pred = np.empty(n)
y= 1 #dummy
for i,x in enumerate(x_train):
    phi_test = rfquery_vector(x)
    rf_pred[i] = np.dot(phi_test,rf_w)   
    c_vector = cquery_vector(x_train, x)
    c_w = inv_ckernel_with_noise @ c_vector
    c_pred[i] = np.dot(z_train,c_w)
    print(rf_pred[i],z_train[i],c_pred[i])
print('rf_error',la.norm(rf_pred-z_test),'kernel error',la.norm(c_pred-z_test))

2.622723650244918 1.0 0.31195455217890866
2.5224840753717626 1.0648477732949493 0.09192711314129598
2.3371699336430893 1.1339007802912116 0.038647253663814496
2.0915892897336548 1.2074317210305021 0.4191227698377169
1.8098714285742599 1.2857309795450185 1.36790049969866
1.5149152074276948 1.369107770624847 2.898780251838879
1.2278947457334954 1.4578913609506803 4.92156660447656
0.967831897241787 1.5524323694142752 7.263191268843482
0.7512431848827568 1.6531041517617928 9.692258659632675
0.59186697774759 1.760304275028181 11.945820378376903
0.5004746951703503 1.8744560875853382 13.756988333696851
0.4847677804616471 1.99601039100441 14.88186765943278
0.5493601421993499 2.125447220334627 15.124243320196857
0.6958437595914835 2.2632777398292667 14.356498413984752
0.9229332343878545 2.4100462616052205 12.535382958419078
1.2266832869288482 2.5663323952081356 9.711488954366736
1.6007715767976762 2.7327533365720766 6.031614071063075
2.0368378137990426 2.909966305413119 1.7335992353851282
2.524

In [193]:
np.set_printoptions(threshold=np.inf)
estimate_k = phi @ phi.T
estimate_ck = Z @ Z.T
dif_k = np.abs(kernel- estimate_k)
dif_ck = np.abs(c_kernel- estimate_ck)
print(dif_k)
mean_dif_k = np.mean(dif_k < .1)
mean_dif_ck = np.mean(dif_ck < .1)

print(f'Mean diff kernel: {mean_dif_k}, Mean diff compound kernel {mean_dif_ck}')
# for i in range(n):
#     for j in range(n):
#         if dif_ck[i][j] > .1:
#             print('kernel:   ', i, j, kernel[i][j], estimate_k[i][j], y_train[i]@y_train[j].T);
#             print('c_kernel: ', i, j, c_kernel[i][j], estimate_ck[i][j])

[[4.44089210e-16 1.79350500e-05 7.17247666e-05 1.61322862e-04
  2.86652229e-04 4.47605005e-04 6.44042658e-04 8.75796093e-04
  1.14266579e-03 1.44442196e-03 1.78080474e-03 2.15152438e-03
  2.55626148e-03 2.99466726e-03 3.46636380e-03 3.97094441e-03
  4.50797387e-03 5.07698883e-03 5.67749817e-03 6.30898338e-03
  6.97089897e-03 7.66267294e-03 8.38370718e-03 9.13337800e-03
  9.91103656e-03 1.07160095e-02 1.15475993e-02 1.24050850e-02
  1.32877228e-02 1.41947464e-02 1.51253679e-02 1.60787782e-02
  1.70541478e-02 1.80506275e-02 1.90673488e-02 2.01034249e-02
  2.11579513e-02 2.22300065e-02 2.33186525e-02 2.44229361e-02
  2.55418893e-02 2.66745299e-02 2.78198627e-02 2.89768800e-02
  3.01445625e-02 3.13218801e-02 3.25077929e-02 3.37012515e-02
  3.49011984e-02 3.61065688e-02 3.73162908e-02 3.85292873e-02
  3.97444759e-02 4.09607703e-02 4.21770810e-02 4.33923162e-02
  4.46053825e-02 4.58151863e-02 4.70206339e-02 4.82206330e-02
  4.94140933e-02 5.05999273e-02 5.17770516e-02 5.29443869e-02
  5.4100

In [ ]:
#print(f'{z_test=}{rf_pred=}{c_pred=}{rf_sigma=}{kernel_sigma=}') does not work for some reason
print(f'z_test={z_test} \nrf_pred={rf_pred} \nc_pred={c_pred}\n\
rf_sigma={rf_sigma} \nkernel_sigma={kernel_sigma}')
rf_mu_time = rf_end - rf_start
c_mu_time = ckernel_end - ckernel_start
print('rf_mu_time, c_mu_time', rf_mu_time, c_mu_time)

rf_sigma_time = rfs_end - rfs_start
c_sigma_time = ckernels_end - ckernels_start
print('rf_sigma_time, c_sigma_time', rf_sigma_time, c_sigma_time)